## Install the neccessary dependencies

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-az79cf4r
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-az79cf4r
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 14.3 MB/s 
     |████████████████████████████████| 101 kB 10.2 MB/s 
     |████████████████████████████████| 6.6 MB 64.7 MB/s 
  Created wheel for transformers: filename=transformers-4.21.0.dev0-py3-none-any.whl size=4447017 sha256=61fc1773e150667ee672e7266d8aebc3af0ea8e56bc1f9820c91f1aa7f6ebc0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-dn0renyt/wheels/b9/f5/17/554bf0fb3fc6b3921def9c39b83cd1bf8576649f1dd985e8aa
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installa

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 14.2 MB/s 


In [ ]:
! pip install chardet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install ipywidgets --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My\ Drive/NLP_Project

## Switch to GPU

In [ ]:
import torch
from tqdm.notebook import tqdm
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda:0'

## Read the original files

In [ ]:
sentences_vi = []
with open("./data/augmented/back_translation/back_translation_data.txt", encoding = 'utf-8') as f:
  for line in f.readlines():
        sentences_vi.append(line)

In [ ]:
# print(len(sentences_en))
print(len(sentences_vi))

65000


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")


In [ ]:
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-vi-en-base")
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-vi-en-base")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo

In [ ]:
src = ["Indonesia phỏng đoán nguyên nhân tàu ngầm chở 53 người mất tích bí ẩn"
tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
model.eval()
summary_ids = model.generate(
                    tokenized_text,
                    max_length=512, 
                    num_beams=5,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True
                )
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)


SyntaxError: ignored

In [ ]:
import math
batch_size = 16
batches = []
for i in range(math.ceil(len(sentences_vi) / batch_size)):
    if i == math.ceil(len(sentences_vi) / batch_size):
        batches.append(sentences_vi[i * batch_size:])
    else:
        batches.append(sentences_vi[i * batch_size: (i+1) * batch_size])

batches = [[sent.strip() for sent in batch] for batch in batches]

In [ ]:
batches[0]

['cách để đi',
 'họ đã xét nghiệm máu cho cheng nhưng mọi thứ vẫn hoàn toàn bình thường',
 'anh có thể gọi tôi không',
 'có rất nhiều yếu tố may rủi ở đây',
 'ai là chúa nói dối',
 'có cửa hàng tiện lợi ở sân bay không',
 'anh đổi ngoại tệ được không',
 'cô ấy mua hai mươi trái chôm chôm và ba con cá',
 'đứa bé có ói òng ọc ra sữa hoặc nước và bé chưa đến mười tuần tuổi hoặc có biểu hiện mất nước không',
 'tôi có thể viết tên địa chỉ số điện thoại của công ty bảo hiểm của anh được không',
 'đừng lo tôi vừa mới đến rồi',
 'mười phút nữa vào giờ rưỡi',
 'tôi có thể hẹn cho chiều mai không',
 'anh rửa hình kỹ thuật số ra được không',
 'tôi cố thu xếp thôi',
 'tôi không dùng ma túy']

In [ ]:
import time
from tqdm import trange

start = time.time()
with open('back_translation_eng.txt', encoding = 'utf-8', mode = 'a') as f:
  pbar = tqdm(total= len(batches))
  for i in range(len(batches)):
    # if i >= 3112:
    sentences = batches[i]
    #Tokenize our input sentence using the model's custom tokenizer
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, max_length=512, truncation=True).to(device)
    translated = model.generate(**inputs.to(device),  max_length=512, 
                    num_beams=3,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True)
    trans = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    # print(trans)
    for translation in trans:
      f.write(translation + '\n')
    pbar.update(1)
print(time.time() - start)  

  0%|          | 0/4063 [00:00<?, ?it/s]

1706.7108855247498


In [ ]:
49792 / 16

3112.0

In [ ]:
with open('back_translation_english.txt', encoding = 'utf-8', mode = 'r') as f:
  a = f.readlines()[:49792]
print(len(a))
print(a[-1])

with open('back_translation_eng.txt', encoding = 'utf-8', mode = 'w') as f:
  for line in a:
    f.write(line)


49792
China adopted the United Nations Framework Agreement on Tobacco Control on 11 October 2005, and soon issued a series of laws banning smoking in rooms in public places in March 2011.



In [10]:
augmented_en = '.../.../.../data/processed/augmented/back_translation/back_translation_data.txt'
with open(augmented_en, encoding = 'utf-8', mode = 'r') as f:
  print(f)

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Since we only perform augmentation on half of the original corpus. We will choose randomly 50% of the sentences and save it to 2 separate files (1 for Vietnamese and 1 for English) to use for augmentation

In [ ]:
import random
random.seed(42)
chosen_index = []
while chosen_lines <= 65000:
  index = random.randint(0,len(sentences_en))
  if index not in chosen_index:
    chosen_index.append(index)
    with open('augment_backtr_vi.txt', mode = 'a', newline = "", encoding = 'utf-8') as f:
      f.write(sentences_vi[index] )
    with open('augment_backtr_en.txt', mode = 'a', newline = "", encoding = 'utf-8') as f:
      f.write(sentences_en[index] )

In [ ]:
augment_backtr = []
with open('augment_backtr_en.txt', encoding = 'utf8') as f:
    for line in f.readlines():
        augment_backtr.append(line)


In [ ]:
len(augment_backtr)

65001

## EN-FR translation

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

### Download the EN-FR model and its tokenizer

In [ ]:
model_marian = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

In [ ]:
model_marian.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(59514, 512, padding_idx=59513)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(59514, 512, padding_idx=59513)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
   

In [ ]:
tokenizer_marian = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

### The MarianMT model requires adding a prefix to the sentence to denote the target language. Here it is French

In [ ]:
src_en = [ (">>fra<< " + sentence) for sentence in augment_backtr ]

In [ ]:
print(src_en[11740])

>>fra<< If we can put our anger inside an engine , it can drive us forward , it can get us through the dreadful moments and it can give us real inner power .



### Divide our data into batches for inference

In [ ]:
import math
batch_size = 8
batches = []
for i in range(math.ceil(len(src_en) / batch_size)):
    if i == math.ceil(len(src_en) / batch_size):
        batches.append(src_en[i * batch_size:])
    else:
        batches.append(src_en[i * batch_size: (i+1) * batch_size])


In [ ]:
len(batches)

8126

### Translate the sentences and store it in the file france_translation.txt

In [ ]:
#4h35p 124, 134, 11668, 48474
import time
from tqdm import trange

start = time.time()
with open('france_translation.txt', encoding = 'utf-8', mode = 'a') as f:
  pbar = tqdm(total= len(batches))
  for i in range(len(batches)):
    sentences = batches[i]
     #Tokenize our input sentence using the model's custom tokenizer
    inputs = tokenizer_marian(sentences, return_tensors="pt", padding=True, max_length=512, truncation=True).to(device)
    translated = model_marian.generate(**inputs.to(device))
    trans = [tokenizer_marian.decode(t, skip_special_tokens=True) for t in translated]
    for translation in trans:
      f.write(translation + '\n')
    pbar.update(1)
print(time.time() - start)  

  0%|          | 0/8126 [00:00<?, ?it/s]

11261.03185224533


In [ ]:
fr_trans = []
with open('france_translation.txt', encoding = 'utf-8') as f:
    for line in f.readlines():
        fr_trans.append(line)

In [ ]:
print(fr_trans[48471])
print(src_en[48471])

Quelques chercheurs ont demandé à quelques centaines de personnes de lire un article scientifique.

>>fra<< A couple of researchers asked a few hundred people to read a scientific article .



## FR - EN translation

### Download the FR-EN model and its tokenizer

In [ ]:
model_backtr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

In [ ]:
model_backtr.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(59514, 512, padding_idx=59513)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(59514, 512, padding_idx=59513)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
   

In [ ]:
tokenizer_backtr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

### Add prefix to our sentences like above. This time, our target language is english, so the prefix is ">>eng<<"

In [ ]:
src_fr = [ (">>eng<< " + sentence) for sentence in fr_trans ]

In [ ]:
src_fr[9627]

'>>eng<< Trois jambes, trois jambes, trois jambes, mais dans la nature, les insectes ont souvent perdu leurs jambes.\n'

### Divide our data into small batches for inference

In [ ]:
batch_size = 8
batches = []
for i in range(math.ceil(len(src_fr) / batch_size)):
    if i == math.ceil(len(src_fr) / batch_size):
        batches.append(src_fr[i * batch_size:])
    else:
        batches.append(src_fr[i * batch_size: (i+1) * batch_size])


In [ ]:
len(batches)

8126

### Generate FR-EN translations

In [ ]:
import time
start = time.time()
with open('english_backtranslations.txt',encoding = 'utf-8', mode = 'a') as f:
  pbar = tqdm(total= len(batches))
  for i in range(len(batches)):
    sentences = batches[i]

    inputs = tokenizer_backtr(sentences, return_tensors="pt", padding=True, max_length = 512, truncation = True).to(device)
    translated = model_backtr.generate(**inputs)
    trans = [tokenizer_backtr.decode(t, skip_special_tokens=True) for t in translated]
    for translation in trans:
      f.write(translation + '\n')
    pbar.update(1)


  0%|          | 0/8126 [00:00<?, ?it/s]